This example demonstrates basic Ray Tune random search and grid search.

In [ ]:
import ray
import time
from ray import tune

In [ ]:
ray.init(address="ray://ray-head:10001")

In [ ]:
def evaluation_fn(step, width, height):
    time.sleep(0.1)
    return (0.1 + width * step / 100)**(-1) + height * 0.1


def easy_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluation_fn(step, width, height)
        # Feed the score back back to Tune.
        tune.report(iterations=step, mean_loss=intermediate_score)

This will do a grid search over the `activation` parameter. This means that each of the two values (`relu` and `tanh`) will be sampled once for each sample (`num_samples`). We end up with 2 * 50 = 100 samples. The `width` and `height` parameters are sampled randomly. `steps` is a constant parameter.

In [ ]:
SMOKE_TEST = True

analysis = tune.run(
        easy_objective,
        metric="mean_loss",
        mode="min",
        num_samples=5 if SMOKE_TEST else 50,
        config={
            "steps": 5 if SMOKE_TEST else 100,
            "width": tune.uniform(0, 20),
            "height": tune.uniform(-100, 100),
            "activation": tune.grid_search(["relu", "tanh"])
        }
)

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)